In [4]:
import numpy as np
import math

In [5]:
#fi - ugao oko X-ose
#teta - ugao oko Y-ose
#psi - ugao oko Z-ose
#funkcija kojom na osnovu uglova rotacije dobijamo matricu izometrije
def Euler2A(fi, teta, psi):
    
    Rx = np.eye(3, dtype = 'float')
    Ry = np.eye(3, dtype = 'float')
    Rz = np.eye(3, dtype = 'float')
    
    Rx[1][1] = math.cos(fi)
    Rx[1][2] = -math.sin(fi)
    Rx[2][1] = math.sin(fi)
    Rx[2][2] = math.cos(fi)
    
    Ry[0][0] = math.cos(teta)
    Ry[2][0] = -math.sin(teta)
    Ry[0][2] = math.sin(teta)
    Ry[2][2] = math.cos(teta)
    
    Rz[0][0] = math.cos(psi)
    Rz[0][1] = -math.sin(psi)
    Rz[1][0] = math.sin(psi)
    Rz[1][1] = math.cos(psi)
    
    A = np.dot(Rz, np.dot(Ry, Rx))
    
    return A

In [152]:
def normalize(v):
    
    norm = np.linalg.norm(v)
    
    if(norm == 0):
        return v
    else:
        return v / norm

In [157]:
#funkcija kojom izometriju A predstavljamo rotacijom oko prave p za ugao fi
def AxisAngle(A):
    
    Ap = A - np.eye(3, dtype='float')

    if(Ap[0][0] == Ap[1][0] and Ap[0][1] == Ap[1][1] and Ap[0][2] == Ap[1][2]):
        p = np.cross(Ap[0], Ap[2])
    else:
        p = np.cross(Ap[0], Ap[1])
        
    #racunanje normalnog vektora 
    u = [0, 0, 0]
    u[0] = p[1]
    u[1] = -p[0]

    u = np.array(u)
    u = normalize(u)
    
    up = np.matmul(A, u)

    fi = math.acos(np.dot(u, up))

    mes_proizvod = np.array([[u[0], u[1], u[2]], [up[0], up[1], up[2]], [p[0], p[1], p[2]]])
    
    if (np.linalg.det(mes_proizvod) < 0):
        p = -1 * p

    p = normalize(p)

    return p, fi

In [142]:
#funkcija koja racuna matricu izometrije Rodrigezovom formulom
def Rodrigez(p, fi):
    
    Rp = np.zeros((3, 3), dtype='float')
    
    for i in range(3):
        for j in range(3):
            Rp[i][j] = p[j] * p[i]
            
    Rp = Rp + math.cos(fi) * (np.eye(3, dtype='float') - Rp)
    
    #pravljenje px matrice
    px = np.zeros((3,3), dtype='float')
    
    px[0][1] = -p[2]
    px[0][2] = p[1]
    px[1][0] = p[2]
    px[1][2] = -p[0]
    px[2][0] = -p[1]
    px[2][1] = p[0]
    
    Rp = Rp + math.sin(fi) * px
            
    return Rp

In [143]:
#funkcija kojom od matrice izometrije dobijamo Ojlerove uglove
def A2Euler(A):
    
    fi = 0
    teta = 0
    psi = 0
    #print(np.linalg.det(np.dot(A, np.transpose(A))))
    
    if(A[2][0] < 1):
        if(A[2][0] > -1):
            #jedinstveno resenje
            psi = np.arctan2(A[1][0], A[0][0])
            teta = np.arcsin(-A[2][0])
            fi = np.arctan2(A[2][1], A[2][2])
            
        else:
            #nije jedinstveno, slucaj Ox3 = -Oz
            psi = np.arctan2(-A[0][1], A[1][1])
            teta = math.PI / 2
            fi = 0
    else:
        #nije jedinstveno, slucaj Ox3 = Oz
        psi = np.arctan2(-A[0][1], A[1][1])
        teta = math.PI / 2
        fi = 0

    return fi, teta, psi

In [144]:
#funkcija racunanja kvaterniona koji predstavlja rotaciju
def AxisAngle2Q(p, fi):
    
    w = math.cos(fi / 2)
    
    #normalizacija
    norm = np.linalg.norm(p)
    
    if norm != 0:
        p = p / norm
        
    q = np.zeros(4, dtype='float')
    
    for i in range(3):
        q[i] = math.sin(fi / 2) * p[i]
    
    q[3] = w
    
    return q

In [145]:
#funkcija kojom od kvaterniona dobijamo vektor i ugao rotacije
def Q2AxisAngle(q):
    
    p = np.zeros(3, dtype='float')
    
    norm = np.linalg.norm(q)
    
    if norm != 0:
        q = q / norm
        
    fi = 2 * np.arccos(q[3])
    
    if(abs(q[3]) == 1):
        
        p[0] = 1
        
    else:
        for i in range(3):
            p[i] = q[i]
            
        norm = np.linalg.norm(p)
    
        if norm != 0:
            p = p / norm
            
    return p, fi

In [165]:
print("-------------------------------Dati test primer---------------------------------")
print("================================================================================\n\n")

fi = -math.atan(1/4)
teta =  -math.asin(8/9)
psi = math.atan(4)

print("Pocetni uglovi:")
print(np.round(fi, 4), np.round(teta, 4), np.round(psi, 4), "\n")

A = Euler2A(fi, teta, psi)

print("Rezultat Euler2A funkcije:\n", np.round(A, 4), "\n")

axis, angle = AxisAngle(A)

print("Rezultat AxisAngle funkcije:")
print(np.round(axis, 5), np.round(angle, 5), "\n")

ARod = Rodrigez(np.array([0.33333, -0.66667, 0.66667]), 1.5708)

print("Rezultat formule Rodrigeza:\n", np.round(ARod, 4), "\n")

[fi1, teta1, psi1] = A2Euler(A)
print("Rezultat A2Euler funkcije:")
print(np.round(fi1, 4), np.round(teta1, 4), np.round(psi1, 4), "\n")

q = AxisAngle2Q(np.array([0.33333, -0.66667, 0.66667]), 1.5708)
print("Rezultat AngleAxis2Q funkcije:\n", np.round(q, 4), "\n")

axis1, angle1 = Q2AxisAngle(q)

print("Rezultat Q2AxisAngle funkcije:")
print(np.round(axis1, 5), np.round(angle1, 5), "\n")

print("\n================================================================================")

-------------------------------Dati test primer---------------------------------


Pocetni uglovi:
-0.245 -1.0949 1.3258 

Rezultat Euler2A funkcije:
 [[ 0.1111 -0.8889 -0.4444]
 [ 0.4444  0.4444 -0.7778]
 [ 0.8889 -0.1111  0.4444]] 

Rezultat AxisAngle funkcije:
[ 0.33333 -0.66667  0.66667] 1.5708 

Rezultat formule Rodrigeza:
 [[ 0.1111 -0.8889 -0.4444]
 [ 0.4444  0.4444 -0.7778]
 [ 0.8889 -0.1111  0.4444]] 

Rezultat A2Euler funkcije:
-0.245 -1.0949 1.3258 

Rezultat AngleAxis2Q funkcije:
 [ 0.2357 -0.4714  0.4714  0.7071] 

Rezultat Q2AxisAngle funkcije:
[ 0.33333 -0.66667  0.66667] 1.5708 




In [166]:
print("----------------------------Originalni test primer------------------------------")
print("================================================================================\n\n")

fi = -math.atan(1/5)
teta =  -math.asin(7/9)
psi = math.atan(5)

print("Pocetni uglovi:")
print(np.round(fi, 4), np.round(teta, 4), np.round(psi, 4), "\n")

A = Euler2A(fi, teta, psi)

print("Rezultat Euler2A funkcije:\n", np.round(A, 4), "\n")

axis, angle = AxisAngle(A)

print("Rezultat AxisAngle funkcije:")
print(np.round(axis, 5), np.round(angle, 5), "\n")

ARod = Rodrigez(axis, angle)

print("Rezultat formule Rodrigeza:\n", np.round(ARod, 4), "\n")

[fi1, teta1, psi1] = A2Euler(A)
print("Rezultat A2Euler funkcije:")
print(np.round(fi1, 4), np.round(teta1, 4), np.round(psi1, 4), "\n")

q = AxisAngle2Q(axis, angle)
print("Rezultat AngleAxis2Q funkcije:\n", np.round(q, 4), "\n")

axis1, angle1 = Q2AxisAngle(q)

print("Rezultat Q2AxisAngle funkcije:")
print(np.round(axis1, 5), np.round(angle1, 5), "\n")

print("\n================================================================================")

----------------------------Originalni test primer------------------------------


Pocetni uglovi:
-0.1974 -0.8911 1.3734 

Rezultat Euler2A funkcije:
 [[ 0.1233 -0.9316 -0.3419]
 [ 0.6163  0.3419 -0.7094]
 [ 0.7778 -0.1233  0.6163]] 

Rezultat AxisAngle funkcije:
[ 0.29331 -0.56029  0.77462] 1.53004 

Rezultat formule Rodrigeza:
 [[ 0.1233 -0.9316 -0.3419]
 [ 0.6163  0.3419 -0.7094]
 [ 0.7778 -0.1233  0.6163]] 

Rezultat A2Euler funkcije:
-0.1974 -0.8911 1.3734 

Rezultat AngleAxis2Q funkcije:
 [ 0.2031 -0.388   0.5365  0.7214] 

Rezultat Q2AxisAngle funkcije:
[ 0.29331 -0.56029  0.77462] 1.53004 


